<a href="https://colab.research.google.com/github/chhak/DeepLearning/blob/master/Chatbot_create_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#구글 드라이브 연동
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
4/4AHA7xk34hbwrx-Bs3tb0Gh7msgnCExO6b_0OfngaaIvXBY7N8VDZZU
Mounted at /content/drive


In [30]:
# 날짜 : 2020/09/10
# 이름 : 김철학
# 내용 : LSTM 챗봇 실습하기
# 출처 : https://wikidocs.net/45101

import codecs
import tensorflow as tf
import keras
import numpy as np
import random, sys
from keras.models import Sequential
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.layers import Dense, Activation, Dropout, Embedding
from keras.layers import LSTM
from keras.optimizers import RMSprop
from keras.utils.data_utils import get_file
from keras.models import load_model
from bs4 import BeautifulSoup

In [3]:
# 파일 로드 함수정의
def load_data(file):
  result = []

  with open(file, 'r', encoding='utf-8') as f:
    lines = f.read().splitlines()

    for line in lines:
      data = line.split(',')
      data = data[0] + ' ' + data[1] + '\n'
      result.append(data)

  result = result[1:] # header정보 제외
  return result

In [ ]:
# 데이터 로드
chat_dataset = load_data('/content/drive/My Drive/Tensorflow_works/data/ChatbotData.csv')
text = ' '.join(chat_dataset)
print(text)

In [9]:
# 문자 토큰화
tokenizer = Tokenizer()
tokenizer.fit_on_texts([text])
voca_size = len(tokenizer.word_index) + 1

print(tokenizer.word_index)

{'거예요': 1, '수': 2, '너무': 3, '더': 4, '좋아하는': 5, '거': 6, '잘': 7, '것': 8, '많이': 9, '안': 10, '좋은': 11, '게': 12, '좀': 13, '사람': 14, '같아요': 15, '있어요': 16, '싶어': 17, '있을': 18, '같아': 19, '사람이': 20, '나': 21, '마세요': 22, '건': 23, '해보세요': 24, '내가': 25, '내': 26, '마음이': 27, '이제': 28, '썸': 29, '어떻게': 30, '왜': 31, '다': 32, '그': 33, '다른': 34, '싶다': 35, '있는': 36, '이별': 37, '다시': 38, '수도': 39, '시간이': 40, '정말': 41, '또': 42, '오늘': 43, '좋을': 44, '하고': 45, '없어': 46, '하는': 47, '것도': 48, '같이': 49, '할': 50, '있어': 51, '자꾸': 52, '될': 53, '없어요': 54, '걸': 55, '바랄게요': 56, '한': 57, '제가': 58, '진짜': 59, '돼요': 60, '저도': 61, '먼저': 62, '해': 63, '있을까': 64, '헤어진지': 65, '그런': 66, '나를': 67, '좋아요': 68, '마음을': 69, '보세요': 70, '될까': 71, '여자친구가': 72, '계속': 73, '없는': 74, '혼자': 75, '남자친구가': 76, '때': 77, '못': 78, '드세요': 79, '먹고': 80, '않아요': 81, '좋죠': 82, '하세요': 83, '마음': 84, '힘든': 85, '바랍니다': 86, '그럴': 87, '일이': 88, '연락': 89, '해요': 90, '연애': 91, '좋아': 92, '말해보세요': 93, '뭐': 94, '생각해요': 95, '사랑이': 96, '좋겠어요': 97, '사랑': 98, '친구가': 99, '

In [22]:
# 학습데이터 생성
sequences = []

for line in text.split('\n'):
  encoded = tokenizer.texts_to_sequences([line])[0]

  for i in range(1, len(encoded)):
    seq = encoded[:i+1]
    sequences.append(seq)

sequences

[[9445, 9446],
 [9445, 9446, 1994],
 [9445, 9446, 1994, 42],
 [9445, 9446, 1994, 42, 2896],
 [9447, 577],
 [9447, 577, 976],
 [9447, 577, 976, 977],
 [9447, 577, 976, 977, 2897],
 [5179, 1716],
 [5179, 1716, 35],
 [5179, 1716, 35, 2407],
 [5179, 1716, 35, 2407, 443],
 [5179, 1716, 35, 2407, 443, 82],
 [5179, 829],
 [5179, 829, 1716],
 [5179, 829, 1716, 35],
 [5179, 829, 1716, 35, 2407],
 [5179, 829, 1716, 35, 2407, 443],
 [5179, 829, 1716, 35, 2407, 443, 82],
 [9448, 5180],
 [9448, 5180, 5181],
 [9448, 5180, 5181, 5182],
 [5183, 9449],
 [5183, 9449, 38],
 [5183, 9449, 38, 1092],
 [5183, 9449, 38, 1092, 314],
 [5183, 9449, 38, 1092, 314, 12],
 [5183, 9449, 38, 1092, 314, 12, 84],
 [5183, 9449, 38, 1092, 314, 12, 84, 5184],
 [5183, 139],
 [5183, 139, 38],
 [5183, 139, 38, 1092],
 [5183, 139, 38, 1092, 314],
 [5183, 139, 38, 1092, 314, 12],
 [5183, 139, 38, 1092, 314, 12, 84],
 [5183, 139, 38, 1092, 314, 12, 84, 5184],
 [978, 9450],
 [978, 9450, 31],
 [978, 9450, 31, 9451],
 [978, 9450, 3

In [23]:
# 학습데이터 패딩(길이 일치)
pad_sequences = pad_sequences(sequences, maxlen=20, padding='pre')
pad_sequences

TypeError: ignored

In [28]:
# 학습, 라벨 분리
pad_sequences = np.array(pad_sequences)
train_x = pad_sequences[:,:-1]
train_y = pad_sequences[:,-1]

train_x[0]

IndexError: ignored

In [25]:
train_y = to_categorical(train_y, num_classes=voca_size)
train_y[1]

array([0., 0., 0., ..., 0., 0., 0.], dtype=float32)

In [31]:
# 모델 구성
model = Sequential()
model.add(Embedding(voca_size, 10, input_length=20))
model.add(LSTM(128))
model.add(Dense(voca_size, activation='softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['acc'])

In [32]:
# 모델 학습
model.fit(train_x, 
          train_y, 
          batch_size=128, 
          epochs=10)

model

Epoch 1/10
578/578 [==============================] - 10s 17ms/step - loss: 8.9307 - acc: 0.0138
Epoch 2/10
578/578 [==============================] - 10s 16ms/step - loss: 8.3575 - acc: 0.0211
Epoch 3/10
578/578 [==============================] - 10s 17ms/step - loss: 8.0438 - acc: 0.0302
Epoch 4/10
578/578 [==============================] - 10s 17ms/step - loss: 7.7728 - acc: 0.0336
Epoch 5/10
578/578 [==============================] - 10s 16ms/step - loss: 7.5210 - acc: 0.0372
Epoch 6/10
578/578 [==============================] - 10s 17ms/step - loss: 7.2804 - acc: 0.0426
Epoch 7/10
578/578 [==============================] - 10s 17ms/step - loss: 7.0519 - acc: 0.0475
Epoch 8/10
578/578 [==============================] - 9s 16ms/step - loss: 6.8370 - acc: 0.0523
Epoch 9/10
578/578 [==============================] - 9s 16ms/step - loss: 6.6335 - acc: 0.0580
Epoch 10/10
578/578 [==============================] - 10s 17ms/step - loss: 6.4391 - acc: 0.0647


In [ ]:
# 모델 저장
model.save('/content/drive/My Drive/Tensorflow_works/model/chatbot.model')

Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Assets written to: /content/drive/My Drive/Tensorflow_works/model/chatbot3.model/assets
